# **Custom YOLOv5**
#### **Stefano Binotto** (*matr*. 2052421) with the special contribution of **Edoardo Bastianello** (*matr*. 2053077) for the shooting of the images to include in the datasets on which we trained and validated the model.

The labeling was performed using [Make Sense](https://www.makesense.ai/).
 

## 1. Setup

Clone the official repository, install all the necessary dependencies and load the datasets.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone

# install all the dependencies
%cd yolov5 # move to the directory containing "requirements.txt"
%pip install -qr requirements.txt  # install

# importing necessary modules
import torch
import utils

display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-304-g51fb467 Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.2/78.2 GB disk)


Load and unzip all the datasets.

In [ ]:
# mount my personal Google Drive "drive" folder 
# in order to speed up the loading of the images 
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# unzip trainingset and testset
!unzip -q ../drive/MyDrive/train_data.zip -d ../
!unzip -q ../drive/MyDrive/test.zip -d ../

## 2. Train on Custom Dataset

Training the model on the dataset I uploaded on my Drive folder. 

In order to perform transfer learning I had to rewrite the configuration file "***custom_data.yaml***" to specify the location of the training set and the classes we want the model to be able to predict.
The best model is saved in "runs/train/exp/weights/best.pt", while the model of the last epoch is saved in "runs/train/exp/weights/last.pt".

The pre-trained weigths we used as initialization are the one related to the [YOLOV5m.pt](https://github.com/ultralytics/yolov5/wiki/Tips-for-Best-Training-Results#:~:text=for%20background%20images.-,Model%20Selection,-Larger%20models%20like) model, which has about 21 million parameters, which makes it a very accurate model but still quite light.

- Custom Trainingset: [link](https://drive.google.com/file/d/1B4DFxi3NhfrCJVFZX9WnrT5qizKkck9K/view?usp=sharing)

In [ ]:
# Fine tune YOLOv5s on custom dataset for 60 epochs, 5 mini-batch size
!python train.py --img 640 --batch 5 --epochs 60 --data custom_data.yaml --weights yolov5m.pt --cache

train: weights=yolov5m.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=60, batch_size=5, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-304-g51fb467 Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

## 3. Validation
Validate our best model on the validation set I uploaded on my Drive folder. The model used is "runs/train/exp/weights/best.pt", which is the best one found during the training. "***custom_coco_testing.yaml***" is the configuration file I wrote in order to perform the validation on the custom dataset. I specified which dataset to use and the classes we want the model to be able to predict.

In [ ]:
# Run val.py on validationset
!python val.py --weights runs/train/exp/weights/best.pt --data custom_coco_testing.yaml --img 640 --iou 0.65 --half

val: data=/content/yolov5/data/custom_coco_testing.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v6.1-304-g51fb467 Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning '/content/yolov5/../test/labels' images and labels...30 found, 0 missing, 0 empty, 0 corrupt: 100% 30/30 [00:00<00:00, 379.13it/s]
val: New cache created: /content/yolov5/../test/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 1/1 [00:01<00:00,  1.49s/it]
                 all         30         65      0.924      0.985      0.975      0.727
Speed: 0.1ms pre-process, 6.1ms 

## 4. Inference for single image test

Using the best weights achieved so far (the path to the weights in my case was: runs/train/exp/weights/best.pt), the script `detect.py`can run the model inference on a wide range of sources, and save results to the `runs/detect` folder. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image 
                          vid.mp4  # video
                          path/  # directory
                          path/*.jpg  # glob
                          'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                          'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```

In [ ]:
# change "conf" value for different FP tolerance
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ../test/images/

# to use during the test to display the predicted images 
display.Image(filename='runs/detect/exp28/01.jpg', width=600)

detect: weights=['runs/train/exp/weights/best.pt'], source=../test/images/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-304-g51fb467 Python-3.7.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/30 /content/test/images/01.jpg: 384x640 2 hands, Done. (0.024s)
image 2/30 /content/test/images/02.jpg: 384x640 2 hands, Done. (0.023s)
image 3/30 /content/test/images/03.jpg: 384x640 2 hands, Done. (0.023s)
image 4/30 /content/test/images/04.jpg: 384x640 2 hands, Done. (0.019s)
image 5/30 /content/test/images/05.jpg: 384x640 2 hands, Done. (0.019s)
ima

## 5. Export best model

In [ ]:
#export the best model by converting the best weights "best.pt" into an .onnx file
!python export.py --weights runs/train/exp/weights/best.pt --include onnx

export: data=data/coco128.yaml, weights=['runs/train/exp/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v6.1-304-g51fb467 Python-3.7.13 torch-1.11.0+cu102 CPU

Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs

PyTorch: starting from runs/train/exp/weights/best.pt with output shape (1, 25200, 6) (40.3 MB)

TorchScript: starting export with torch 1.11.0+cu102...
TorchScript: export success, saved as runs/train/exp/weights/best.torchscript (80.1 MB)

ONNX: starting export with onnx 1.12.0...
ONNX: export success, saved as runs/train/exp/weights/best.onnx (80.0 MB)

Export complete (12.21s)
Results saved to /content/yolov5/runs/train/exp/weights
Det